# Customizing Your Typechecker

## Configuring Your Typechecker


### Catching dynamic behavior

In [14]:
# The following option flags expressions with variables of 'Any' type

!mypy  --disallow-any-expr src/any_expr.py

src/any_expr.py:5: error: Expression has type "Any"
Found 1 error in 1 file (checked 1 source file)


In [15]:
# The following option flags variable definitions of 'Any' type

!mypy  --disallow-any-generics src/any_var.py

src/any_var.py:1: error: Missing type parameters for generic type "list"
Found 1 error in 1 file (checked 1 source file)


### Requiring types

In [16]:
# The following option flags untyped function definitions

!mypy  --disallow-untyped-defs src/untyped_func.py

src/untyped_func.py:1: error: Function is missing a type annotation
Found 1 error in 1 file (checked 1 source file)


### Handling None/Optional


In [17]:
# The following option flags unhandled None cases

!mypy --strict-optional src/unhandled_none.py

src/unhandled_none.py:5: error: Unsupported operand types for + ("None" and "int")
src/unhandled_none.py:5: note: Left operand is of type "Optional[int]"
Found 1 error in 1 file (checked 1 source file)


In [18]:
# The following option disables the conversion of None values as Optional types

!mypy --no-implicit-optional src/ambiguous_none.py


src/ambiguous_none.py:1: error: Incompatible default for argument "x" (default has type "None", argument has type "int")
Found 1 error in 1 file (checked 1 source file)


### Mypy reporting

In [35]:
# The following option creates a HTML report after typechecking a module or package

!mypy --html-report ./src

usage: mypy [-h] [-v] [-V] [more options; see below]
            [-m MODULE] [-p PACKAGE] [-c PROGRAM_TEXT] [files ...]
mypy: error: Missing target module, package, files, or command.


In [28]:
# The following option tracks explicit Any expressions on a line-by-line basis

!mypy --any-exprs-report ./src


usage: mypy [-h] [-v] [-V] [more options; see below]
            [-m MODULE] [-p PACKAGE] [-c PROGRAM_TEXT] [files ...]
mypy: error: Missing target module, package, files, or command.


## Alternative Typecheckers


### Pyre

You can install it as follows

In [ ]:
"""
!pip install pyre-check
"""

'\n!pip install pyre-check\n'

Pyre allows you to make queries to inspect the codebase

In [ ]:
# The following query returns all the attributes of any class in the codebase

!pyre query "attributes(mypy.errors.CompileError)"

/bin/bash: pyre: command not found


In [ ]:
# The following query returns all the callees of any function:

!pyre query "callees(mypy.errors.remove_path_prefix)"


/bin/bash: pyre: command not found


### Python Static Analyzer (Pysa)

Consider the case where a user creates a new recipe in a filesystem

In [ ]:
import os


def create_recipe():
    recipe = input("Enter in recipe")
    create_recipe_on_disk(recipe)

def create_recipe_on_disk(recipe):
    command = "touch ~/food_data/{}.json".format(recipe)
    return os.system(command)


The input() statement inside the function create_recipe() is dangerous. What if a malicious user enters the following?:

```
carrots; ls ~;
```

It will list the server's entire home directory!!

To address the last issue, pysa provides tools for 'taint analysis', as follows

In [ ]:
# Create a taint.config file with the following contents

"""
{
    sources: [
        {
            name: "UserControlled", 
            comment: "use to annotate user input"
        }
    ],
    sinks: [
        {
            name: "RemoteCodeExecution", 
            comment: "use to annotate execution of code"
        }
    ],
    features: [],
    rules: [
        {
            name: "Possible shell injection", 
            code: 5001,
            sources: [ "UserControlled" ],
            sinks: [ "RemoteCodeExecution" ],
            message_format: "Data from [{$sources}] source(s) may reach [{$sinks}] sink(s)"
        }
    ]
}
"""

'\n{\n    sources: [\n        {\n            name: "UserControlled", \n            comment: "use to annotate user input"\n        }\n    ],\n    sinks: [\n        {\n            name: "RemoteCodeExecution", \n            comment: "use to annotate execution of code"\n        }\n    ],\n    features: [],\n    rules: [\n        {\n            name: "Possible shell injection", \n            code: 5001,\n            sources: [ "UserControlled" ],\n            sinks: [ "RemoteCodeExecution" ],\n            message_format: "Data from [{$sources}] source(s) may reach [{$sinks}] sink(s)"\n        }\n    ]\n}\n'

In [ ]:
# Next, create a general.pysa file with the following contents

"""
# stubs/taint/general.pysa

# model for raw_input
def input(__prompt = ...) -> TaintSource[UserControlled]: ...

# model for os.system
def os.system(command: TaintSink[RemoteCodeExecution]): ...
"""

'\n# stubs/taint/general.pysa\n\n# model for raw_input\ndef input(__prompt = ...) -> TaintSource[UserControlled]: ...\n\n# model for os.system\ndef os.system(command: TaintSink[RemoteCodeExecution]): ...\n'

In [ ]:
# Next, modify the .pyre_configuration to add in your directory to tell Pyre to detect tainted information 

"""
"source_directories": ["."],
"taint_models_path": ["stubs/taint"]
"""

'\n"source_directories": ["."],\n"taint_models_path": ["stubs/taint"]\n'

In [ ]:
# Then, you can run Pyre against the target file

!pyre analyze TARGET_FILE.py

/bin/bash: pyre: command not found
